In [ ]:
from tkinter import *
from tkinter import messagebox
from PIL import Image, ImageTk
import openpyxl
import os
import mysql.connector
import datetime

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Admin12345", 
    database="grocery_db"
)
cursor = conn.cursor()

categories = {
    "Oil": {"Sunflower Oil": 120, "Mustard Oil": 130, "Coconut Oil": 150},
    "Rice": {"Basmati Rice": 90, "Brown Rice": 80},
    "Dal": {"Toor Dal": 85, "Chana Dal": 75, "Masoor Dal": 70},
    "Wheat": {"Whole Wheat": 50, "Multigrain Wheat": 60},
    "Sugar": {"White Sugar": 40, "Brown Sugar": 50}
}
cart = []
shop_name = "Unique Grocery Store"

root = Tk()
root.title("Unique Grocery Store")
root.geometry("1100x750")

login_frame = Frame(root, bg="white")
billing_frame = Frame(root)
view_frame = Frame(root, bd=3, relief=SOLID)

login_frame.place(relwidth=1, relheight=1)

Label(login_frame, text="Login", font=("Times new roman", 18, "bold"), bg="white").pack(pady=10)
Label(login_frame, text="Username:", font=("Times new roman", 12), bg="white").pack()
username_entry = Entry(login_frame, font=("Times new roman", 12))
username_entry.pack(pady=5)

Label(login_frame, text="Password:", font=("Times new roman", 12), bg="white").pack()
password_entry = Entry(login_frame, show="*", font=("Times new roman", 12))
password_entry.pack(pady=5)

def verify_login():
    user = username_entry.get()
    pw = password_entry.get()
    if user == "admin" and pw == "445566":
        messagebox.showinfo("Login Success", "Welcome to Unique Grocery Store!")
        login_frame.place_forget()
        open_billing_interface()
    else:
        messagebox.showerror("Login Failed", "Invalid Username or Password!")

Button(login_frame, text="Login", font=("Times new roman", 12, "bold"), bg="green", fg="white", command=verify_login).pack(pady=15)

def open_billing_interface():
    billing_frame.place(relwidth=1, relheight=1)
    view_frame.place(x=570, y=50, width=450, height=550)

    global cust_name, category_var, product_var, quantity, result_area

    def update_products(*args):
        selected_category = category_var.get()
        product_var.set("")
        product_menu['menu'].delete(0, 'end')
        for product in categories[selected_category]:
            product_menu['menu'].add_command(label=product, command=lambda value=product: product_var.set(value))

    def add_to_cart():
        name = cust_name.get()
        category = category_var.get()
        item = product_var.get()
        qty = quantity.get()

        if name == "":
            messagebox.showerror("Missing", "Enter Customer Name")
            return
        if category == "" or item == "":
            messagebox.showerror("Missing", "Select Category and Product")
            return
        if not qty.isdigit():
            messagebox.showerror("Invalid", "Enter valid quantity")
            return

        price = categories[category][item]
        total_price = int(qty) * price
        cart.append((item, int(qty), price, total_price))

        cursor.execute("INSERT INTO bills (customer_name, product, quantity, price, total) VALUES (%s, %s, %s, %s, %s)",
                       (name, item, int(qty), price, total_price))
        conn.commit()

        update_bill_area()

    def update_bill_area():
        result_area.delete('1.0', END)
        result_area.insert(END, f"\t\U0001F6D2 {shop_name}\n", "header")
        result_area.insert(END, f"\nCustomer Name: {cust_name.get()}\n\n", "bold")
        result_area.insert(END, f"{'Product':20}{'Qty':>5}{'Price':>10}{'Total':>10}\n", "bold")
        result_area.insert(END, "-"*50 + "\n")
        grand_total = 0
        for item, qty, price, total in cart:
            result_area.insert(END, f"{item:20}{qty:>5}{price:>10}{total:>10}\n", "normal")
            grand_total += total
        result_area.insert(END, "-"*50 + "\n")
        result_area.insert(END, f"{'Grand Total':>45}: Rs. {grand_total}\n", "green")

    def clear_cart():
        cart.clear()
        result_area.delete('1.0', END)

    def reset_fields():
        category_var.set("")
        product_var.set("")
        quantity.delete(0, END)

    def export_to_excel():
        if not cart:
            messagebox.showwarning("Empty Cart", "Add items to cart before exporting.")
            return

        export_win = Toplevel(root)
        export_win.title("Export Bill to Excel")
        export_win.geometry("350x150")
        export_win.configure(bg="#f9f9f9")

        default_filename = f"{cust_name.get().strip().replace(' ', '_')}_bill.xlsx"
        Label(export_win, text="Filename:", font=("Arial", 12), bg="#f9f9f9").pack(pady=10)
        filename_entry = Entry(export_win, font=("Arial", 12))
        filename_entry.insert(0, default_filename)
        filename_entry.pack(pady=5, padx=10, fill=X)

        def save_excel():
            filename = filename_entry.get().strip()
            if not filename.endswith(".xlsx"):
                filename += ".xlsx"

            wb = openpyxl.Workbook()
            ws = wb.active
            ws.title = "Bill"

            ws.append([shop_name])
            ws.append(["Customer Name:", cust_name.get()])
            ws.append(["Product", "Quantity", "Price", "Total"])
            for item, qty, price, total in cart:
                ws.append([item, qty, price, total])

            grand_total = sum(x[3] for x in cart)
            ws.append(["", "", "Grand Total", grand_total])

            try:
                wb.save(filename)
                messagebox.showinfo("Success", f"Bill exported as '{filename}'")
                export_win.destroy()
            except Exception as e:
                messagebox.showerror("Error", f"Failed to save file:\n{str(e)}")

        Button(export_win, text="Save", font=("Times new roman", 11, "bold"), bg="green", fg="white", command=save_excel).pack(pady=10)

    try:
        bg_image = Image.open("bg.jpg")
        bg_image = bg_image.resize((540, 750), Image.Resampling.LANCZOS)
        bg_photo = ImageTk.PhotoImage(bg_image)
        canvas = Canvas(billing_frame, width=540, height=750)
        canvas.pack(fill="both", expand=True)
        canvas.create_image(0, 0, image=bg_photo, anchor="nw")
        canvas.image = bg_photo
    except:
        canvas = Canvas(billing_frame, width=540, height=750, bg="#fffbe7")
        canvas.pack(fill="both", expand=True)

    canvas.create_text(270, 30, text=shop_name, font=("Times new roman", 20, "bold italic underline"), fill="darkred")
    canvas.create_rectangle(50, 60, 490, 270, fill="#fffbe7", outline="")

    canvas.create_text(70, 90, text="Customer Name:", anchor="w", font=("Times new roman", 12, "bold"))
    cust_name = Entry(billing_frame, font=("Times new roman", 12))
    canvas.create_window(310, 90, window=cust_name, width=180)

    canvas.create_text(70, 130, text="Select Category:", anchor="w", font=("Times new roman", 12, "bold"))
    category_var = StringVar()
    category_menu = OptionMenu(billing_frame, category_var, *categories.keys(), command=update_products)
    category_menu.config(font=("Times new roman", 11))
    canvas.create_window(310, 130, window=category_menu, width=180)

    canvas.create_text(70, 170, text="Select Product:", anchor="w", font=("Times new roman", 12, "bold"))
    product_var = StringVar()
    product_menu = OptionMenu(billing_frame, product_var, "")
    product_menu.config(font=("Times new roman", 11))
    canvas.create_window(310, 170, window=product_menu, width=180)

    canvas.create_text(70, 210, text="Enter Quantity:", anchor="w", font=("Times new roman", 12, "bold"))
    quantity = Entry(billing_frame, font=("Times new roman", 12))
    canvas.create_window(310, 210, window=quantity, width=180)

    canvas.create_window(120, 320, window=Button(billing_frame, text="Add to Cart", command=add_to_cart, bg="green", fg="white", font=("Times new roman", 12, "bold")), width=90)
    canvas.create_window(260, 320, window=Button(billing_frame, text="Clear Cart", command=clear_cart, bg="red", fg="white", font=("Times new roman", 12, "bold")), width=90)
    canvas.create_window(380, 320, window=Button(billing_frame, text="Reset Fields", command=reset_fields, bg="orange", fg="white", font=("Times new roman", 12, "bold")), width=90)

    result_area = Text(billing_frame, height=15, width=58, bd=3, relief=SOLID, font=("Courier New", 10), bg="#fffacd", fg="black", wrap=WORD)
    canvas.create_window(270, 480, window=result_area)

    result_area.tag_config("header", foreground="blue", font=("Courier New", 13, "bold"))
    result_area.tag_config("green", foreground="green", font=("Courier New", 11, "bold"))
    result_area.tag_config("bold", font=("Courier New", 11, "bold"))
    result_area.tag_config("normal", font=("Courier New", 10))

    canvas.create_window(270, 630, window=Button(billing_frame, text="Export to Excel", command=export_to_excel, bg="blue", fg="white", font=("Times new roman", 11, "bold")), width=200)

    Button(billing_frame, text="View Bills", command=open_date_selector, bg="purple", fg="white", font=("Times New Roman", 11, "bold")).place(x=730, y=610)

    Label(view_frame, text="Billing History", font=("times new roman", 14, "bold"), fg="darkblue").pack(pady=10)
    global bill_list
    bill_list = Text(view_frame, font=("Courier New", 10), width=60, height=28)
    bill_list.pack()
    bill_list.config(state="disabled")

def open_date_selector():
    top = Toplevel(root)
    top.title("Select Date")
    top.geometry("300x150")

    Label(top, text="Enter Date (YYYY-MM-DD):", font=("Times New Roman", 12)).pack(pady=10)
    cal = Entry(top, font=("Times New Roman", 12))
    cal.insert(0, datetime.date.today().isoformat())
    cal.pack(pady=5)

    def fetch_bills_by_date():
        selected_date = cal.get()
        bill_list.config(state="normal")
        bill_list.delete("1.0", END)

        cursor.execute("SELECT * FROM bills")
        rows = cursor.fetchall()

        bill_list.insert(END, f"{'ID':<5}{'Name':<15}{'Product':<20}{'Qty':<5}{'Price':<7}{'Total':<7}\n")
        bill_list.insert(END, f"{'- '*32}\n")
        for row in rows:
            bill_list.insert(END, f"{row[0]:<5}{row[1]:<15}{row[2]:<20}{row[3]:<5}{row[4]:<7}{row[5]:<7}\n")

        bill_list.config(state="disabled")
        top.destroy()

    Button(top, text="View Bills", command=fetch_bills_by_date, font=("Times New Roman", 11, "bold"), bg="green", fg="white").pack(pady=20)

def on_closing():
    conn.close()
    root.destroy()

root.protocol("WM_DELETE_WINDOW", on_closing)
root.mainloop()
